# 서울시 아파트 실거래 분석 (그래프 포함 종합 정리)
업로드하신 `2025서울시부동산실거래가.xlsx`, `Housing.xlsx`를 바탕으로
전처리 → 분포/요약 → 평형(59㎡/84㎡) → 신축/준신축 비교 → 자치구별 상승률 → 단순 예측(라인)까지 한 번에 재현 가능한 노트북입니다.

- 생성일시: 2025-09-17 04:41:24
- 실행 전제: 동일 경로(`/mnt/data`)에 데이터 파일이 존재해야 합니다.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 한글 폰트(설치되어 있으면 자동 적용)
try:
    import koreanize_matplotlib  # noqa: F401
except Exception as e:
    print('koreanize_matplotlib 미적용(옵션):', e)

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_squared_error

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)


## 1. 데이터 로드 및 전처리

In [ ]:
# 파일 경로
real_path = "/mnt/data/2025서울시부동산실거래가.xlsx"
housing_path = "/mnt/data/Housing.xlsx"

# 실거래 데이터 로드
df = pd.read_excel(real_path, sheet_name="data")

# 아파트만 필터링
df_apartment = df[df["건물용도"] == "아파트"].copy()

# 날짜/파생
df_apartment["계약일"] = pd.to_datetime(df_apartment["계약일"], format="%Y%m%d", errors="coerce")
df_apartment["년월"] = df_apartment["계약일"].dt.to_period("M")
df_apartment["년"] = df_apartment["계약일"].dt.year
df_apartment["월"] = df_apartment["계약일"].dt.month

# 건축년도 정리
df_apartment["건축년도"] = df_apartment["건축년도"].fillna(0).astype(int)
df_apartment.loc[df_apartment["건축년도"] < 1900, "건축년도"] = None

# 면적/금액 정리
df_apartment["건물면적(㎡)"] = df_apartment["건물면적(㎡)"].round(2)
df_apartment = df_apartment[df_apartment["물건금액(만원)"] > 0]

df_apartment.head()


## 2. 전체 분포/요약 (박스플롯, 히스토그램)

In [ ]:
overall_avg = df_apartment["물건금액(만원)"].mean().round(0)
median_all = df_apartment["물건금액(만원)"].median().round(0)
q10_all = df_apartment["물건금액(만원)"].quantile(0.1).round(0)
q90_all = df_apartment["물건금액(만원)"].quantile(0.9).round(0)
print("전체 평균:", overall_avg, "| 중앙값:", median_all, "| 하위10%:", q10_all, "| 상위10%:", q90_all)

# 박스플롯
plt.figure(figsize=(6,6))
plt.boxplot(df_apartment["물건금액(만원)"], vert=True)
plt.title("서울시 아파트 매매금액 분포 (박스플롯)")
plt.ylabel("거래금액(만원)")
plt.grid(True)
plt.show()

# 히스토그램
plt.figure(figsize=(10,6))
plt.hist(df_apartment["물건금액(만원)"], bins=50, edgecolor="black")
plt.title("서울시 아파트 매매금액 분포 (히스토그램)")
plt.xlabel("거래금액(만원)")
plt.ylabel("빈도수")
plt.grid(True)
plt.show()


## 3. 자치구×월별 거래건수 / 전체 월별 추이

In [ ]:
monthly_counts = df_apartment.groupby(["자치구명","년월"]).size().reset_index(name="거래건수")
pivot_counts = monthly_counts.pivot(index="자치구명", columns="년월", values="거래건수").fillna(0).astype(int)
pivot_counts.head()


In [ ]:
# 서울 전체 월별 거래건수 추이
total_trend = df_apartment.groupby("년월").size()
plt.figure(figsize=(10,5))
plt.plot(total_trend.index.astype(str), total_trend.values, marker="o")
plt.title("서울시 전체 아파트 월별 거래건수 추이")
plt.ylabel("거래건수")
plt.xlabel("년월")
plt.grid(True)
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()


## 4. 평형대 분석: 59㎡ / 84㎡ (유사 면적 묶음)

In [ ]:
def classify_area(x):
    if 56 <= x <= 62:
        return "59㎡"
    elif 82 <= x <= 86:
        return "84㎡"
    return None

df_apartment["대표면적"] = df_apartment["건물면적(㎡)"].apply(classify_area)
df_target = df_apartment[df_apartment["대표면적"].notnull()].copy()

pivot_area_stats = df_target.groupby(["자치구명","대표면적"])["물건금액(만원)"].mean().unstack()
ax = pivot_area_stats.plot(kind="bar", figsize=(14,8))
plt.title("자치구별 아파트 평균 매매금액 (59㎡ vs 84㎡)")
plt.ylabel("평균 매매금액(만원)")
plt.xlabel("자치구")
plt.xticks(rotation=45, ha="right")
plt.grid(True, axis="y")
plt.tight_layout()
plt.show()


## 5. 59㎡ 자치구별 평균 순위 및 백분위

In [ ]:
rank59 = (
    df_target[df_target['대표면적']=='59㎡']
    .groupby('자치구명')['물건금액(만원)'].mean().round(0).sort_values(ascending=False).reset_index()
)
rank59['순위'] = rank59['물건금액(만원)'].rank(method='dense', ascending=False).astype(int)
rank59.head(10)


In [ ]:
# 59㎡ 기준 10억7천(=107000만원)의 상위 백분위
value = 107000
prices_59 = df_target[df_target["대표면적"]=="59㎡"]["물건금액(만원)"]
percentile = (prices_59 < value).mean()*100
upper = 100 - percentile
print(f"59㎡에서 10억7천만원은 상위 약 {upper:.2f}%")


## 6. 신축(2020~) vs 준신축(2015~2019) 비교 (59㎡/84㎡)

In [ ]:
# 59㎡
prices_59_new = df_target[(df_target['대표면적']=='59㎡') & (df_target['건축년도']>=2020)]['물건금액(만원)']
prices_59_old = df_target[(df_target['대표면적']=='59㎡') & (df_target['건축년도'].between(2015,2019))]['물건금액(만원)']
print('59㎡ 신축 평균/중앙:', prices_59_new.mean().round(0), prices_59_new.median().round(0))
print('59㎡ 준신축 평균/중앙:', prices_59_old.mean().round(0), prices_59_old.median().round(0))

# 84㎡
prices_84_new = df_target[(df_target['대표면적']=='84㎡') & (df_target['건축년도']>=2020)]['물건금액(만원)']
prices_84_old = df_target[(df_target['대표면적']=='84㎡') & (df_target['건축년도'].between(2015,2019))]['물건금액(만원)']
print('84㎡ 신축 평균/중앙:', prices_84_new.mean().round(0), prices_84_new.median().round(0))
print('84㎡ 준신축 평균/중앙:', prices_84_old.mean().round(0), prices_84_old.median().round(0))


In [ ]:
# 자치구별 59㎡: 신축 vs 전체 평균 차이
area59_all = df_target[df_target['대표면적']=='59㎡'].groupby('자치구명')['물건금액(만원)'].mean().round(0).reset_index(name='전체')
area59_new = df_target[(df_target['대표면적']=='59㎡') & (df_target['건축년도']>=2020)].groupby('자치구명')['물건금액(만원)'].mean().round(0).reset_index(name='신축')
cmp59 = pd.merge(area59_all, area59_new, on='자치구명', how='outer').fillna(0)
cmp59['차이'] = cmp59['신축'] - cmp59['전체']

plt.figure(figsize=(14,8))
plt.bar(cmp59['자치구명'], cmp59['차이'], color=['red' if x<0 else 'blue' for x in cmp59['차이']])
plt.axhline(0, color='black', lw=1)
plt.title('자치구별 59㎡ 신축(2020~) vs 전체 평균 매매금액 차이')
plt.ylabel('차이 (만원)')
plt.xlabel('자치구')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()


In [ ]:
# 자치구별 84㎡: 신축 vs 전체 평균 차이
area84_all = df_target[df_target['대표면적']=='84㎡'].groupby('자치구명')['물건금액(만원)'].mean().round(0).reset_index(name='전체')
area84_new = df_target[(df_target['대표면적']=='84㎡') & (df_target['건축년도']>=2020)].groupby('자치구명')['물건금액(만원)'].mean().round(0).reset_index(name='신축')
cmp84 = pd.merge(area84_all, area84_new, on='자치구명', how='outer').fillna(0)
cmp84['차이'] = cmp84['신축'] - cmp84['전체']

plt.figure(figsize=(14,8))
plt.bar(cmp84['자치구명'], cmp84['차이'], color=['red' if x<0 else 'blue' for x in cmp84['차이']])
plt.axhline(0, color='black', lw=1)
plt.title('자치구별 84㎡ 신축(2020~) vs 전체 평균 매매금액 차이')
plt.ylabel('차이 (만원)')
plt.xlabel('자치구')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()


## 7. 자치구별 2025년 상승률 (기준월→최신월) & Top/Bottom10 그래프

In [ ]:
# 2025년만
df_2025 = df_apartment[df_apartment['계약일'].dt.year == 2025]
monthly_avg = (
    df_2025.groupby(['자치구명','년월'])['물건금액(만원)']
    .mean().reset_index().sort_values(['자치구명','년월'])
)

# 기준월/최신월
first_tbl = monthly_avg.sort_values(['자치구명','년월']).drop_duplicates('자치구명', keep='first')[['자치구명','년월']].rename(columns={'년월':'기준년월'})
last_tbl  = monthly_avg.sort_values(['자치구명','년월']).drop_duplicates('자치구명', keep='last')[['자치구명','년월']].rename(columns={'년월':'최신년월'})

first_price = monthly_avg.merge(first_tbl, on='자치구명'); first_price = first_price[first_price['년월']==first_price['기준년월']][['자치구명','년월','물건금액(만원)']].rename(columns={'년월':'기준년월','물건금액(만원)':'기준가(만원)'})
last_price  = monthly_avg.merge(last_tbl, on='자치구명');  last_price  = last_price[last_price['년월']==last_price['최신년월']][['자치구명','년월','물건금액(만원)']].rename(columns={'년월':'최신년월','물건금액(만원)':'최신가(만원)'})
growth = first_price.merge(last_price, on='자치구명')

def p2m(p): y,m = map(int, str(p).split('-')); return y*12+m
growth['상승률(%)'] = ((growth['최신가(만원)']-growth['기준가(만원)'])/growth['기준가(만원)']*100).round(2)
growth['월수'] = growth.apply(lambda r: p2m(r['최신년월'])-p2m(r['기준년월']), axis=1).replace(0,1)
growth['월CAGR(%)'] = (((growth['최신가(만원)']/growth['기준가(만원)'])**(1/growth['월수']))-1).mul(100).round(2)

growth_sorted = growth.sort_values('상승률(%)', ascending=False).reset_index(drop=True)
display(growth_sorted.head())

# Top 10
top10 = growth_sorted.head(10)
plt.figure(figsize=(12,6))
plt.bar(top10['자치구명'], top10['상승률(%)'])
plt.title('2025년 자치구별 매매가격 상승률 Top 10 (기준월 대비 최신월)')
plt.ylabel('상승률(%)')
plt.xticks(rotation=45, ha='right'); plt.grid(True, axis='y'); plt.tight_layout(); plt.show()

# Bottom 10
bottom10 = growth_sorted.tail(10)
plt.figure(figsize=(12,6))
plt.bar(bottom10['자치구명'], bottom10['상승률(%)'])
plt.title('2025년 자치구별 매매가격 상승률 Bottom 10 (기준월 대비 최신월)')
plt.ylabel('상승률(%)')
plt.xticks(rotation=45, ha='right'); plt.grid(True, axis='y'); plt.tight_layout(); plt.show()


## 8. 상위3/하위3 자치구 실제값 + 단순 선형 예측 라인

In [ ]:
from sklearn.linear_model import LinearRegression

def forecast_by_gu(df_gu):
    df_gu = df_gu.sort_values('년월')
    if len(df_gu) < 3:
        return None
    X = np.arange(1, len(df_gu)+1).reshape(-1,1)
    y = df_gu['물건금액(만원)'].values
    model = LinearRegression().fit(X, y)
    future_X = np.arange(len(df_gu)+1, len(df_gu)+3+1).reshape(-1,1)
    y_pred = model.predict(future_X)
    last_period = df_gu['년월'].max().to_timestamp()
    future_months = pd.period_range(last_period + pd.offsets.MonthBegin(1), periods=3, freq='M')
    return pd.DataFrame({'자치구명': df_gu['자치구명'].iloc[0], '년월': future_months, '예측평균가(만원)': np.round(y_pred,0)})

fc_list = []
for gu, sub in monthly_avg.groupby('자치구명'):
    out = forecast_by_gu(sub)
    if out is not None:
        fc_list.append(out)
forecast_df = pd.concat(fc_list, ignore_index=True) if fc_list else pd.DataFrame()

def plot_actual_vs_forecast(gu_name):
    actual = monthly_avg[monthly_avg['자치구명']==gu_name].sort_values('년월')
    future = forecast_df[forecast_df['자치구명']==gu_name].sort_values('년월')
    if len(actual) < 3: 
        return
    plt.figure(figsize=(10,5))
    plt.plot(actual['년월'].astype(str), actual['물건금액(만원)'], marker='o', label='실제 평균가')
    if not future.empty:
        plt.plot(future['년월'].astype(str), future['예측평균가(만원)'], linestyle='--', marker='o', label='예측 평균가')
    plt.title(f'{gu_name} 월별 평균 매매가(만원): 실제값과 예측')
    plt.xlabel('년월'); plt.ylabel('평균 매매가(만원)')
    plt.xticks(rotation=45, ha='right'); plt.grid(True); plt.legend(); plt.tight_layout(); plt.show()

top3 = growth_sorted.head(3)['자치구명'].tolist()
bot3 = growth_sorted.tail(3)['자치구명'].tolist()
for gu in top3 + bot3:
    plot_actual_vs_forecast(gu)
